In [4]:
import pandas as pd
import numpy as np
from preprocessing import Preprocessing
from sklearn.preprocessing import LabelEncoder

# Dataload

In [7]:
# data loading
import time
s= time.time()
datapath = './data/'

pc = Preprocessing(savepath = datapath)
pc.data_to_vectors() # 10분 이상 걸림
pc.save_item_vector() # csv로 저장
print(time.time()-s)


C:\Users\jh_r_\PycharmProjects\recommendation-ecam-ncf\preprocessing.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  self.pdde = pd.DataFrame(pd.read_csv(self.datapath+'LPOINT_BIG_COMP_02_PDDE.csv')).sort_values('cust')


805.9935526847839


In [15]:

item_vector = pd.read_csv(datapath+ 'item.csv',encoding='UTF-8').iloc[:,1:]
user_vector = pd.read_csv(datapath + 'user.csv',encoding='UTF-8').iloc[:,1:]
target_vector = pd.read_csv(datapath + 'target.csv',encoding='UTF-8').iloc[:,1:]

In [19]:
item_vector

,clac_mcls_nm,clac_hlv_nm
0,DVD/홈시어터,35
1,TV,35
2,가공유,40
3,가방세트,56
4,간식/안주형과자,7
...,...,...
344,헤어케어,57
345,호주산소고기,50
346,홈웨어,26
347,홍삼/인삼가공식품,1


In [11]:
def label_encoding(item_vector,user_vector):
    label = LabelEncoder()

    iv = label.fit_transform(item_vector['clac_hlv_nm'])
    item_vector['clac_hlv_nm'] = iv
    
    uv = user_vector
    uv.loc[uv['rct_no'] < 30, 'rct_no'] = 0
    uv.loc[(uv['rct_no'] >=30) & (uv['rct_no'] < 88), 'rct_no'] = 1
    uv.loc[(uv['rct_no'] >=88) & (uv['rct_no'] < 214), 'rct_no'] = 2
    uv.loc[uv['rct_no'] >=214, 'rct_no'] = 3
    uv['ma_fem_dv'] = label.fit_transform(uv['ma_fem_dv'])
    uv['ages'] = label.fit_transform(uv['ages'])
    uv['zon_hlv']=label.fit_transform(uv['zon_hlv'])

    user_vector = uv

    return item_vector, user_vector

i,u = label_encoding(item_vector, user_vector)



In [18]:
u

,cust,rct_no,ma_fem_dv,ages,zon_hlv
0,M057015266,3,0,2,9
1,M919374790,3,0,3,16
2,M287960590,3,1,2,16
3,M763326819,3,0,2,10
4,M124357021,3,0,2,7
...,...,...,...,...,...
26912,M256907327,0,1,0,16
26913,M257378621,0,0,0,16
26914,M911583788,0,1,1,14
26915,M911323403,0,1,5,16


In [13]:
i

,clac_mcls_nm,clac_hlv_nm
0,DVD/홈시어터,35
1,TV,35
2,가공유,40
3,가방세트,56
4,간식/안주형과자,7
...,...,...
344,헤어케어,57
345,호주산소고기,50
346,홈웨어,26
347,홍삼/인삼가공식품,1


In [21]:
rct = pd.Series(user_vector['rct_no'])
rct.describe()
# 30 미만(상위 75%)은 0, 30~88 미만(75~50%)은 1, 88~ 214미만(50~25%)은 2, 214이상(25%이상)은 3


count    26917.000000
mean         1.505145
std          1.116796
min          0.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          3.000000
Name: rct_no, dtype: float64

In [110]:
# rct_no = onehot.fit_transform(uv.iloc[:,[0]]) # 4
# ma_fem_dv = onehot.fit_transform(uv.iloc[:,[1]]) # 2
# ages = onehot.fit_transform(uv.iloc[:,[2]]) # 6
# zon_hlv = onehot.fit_transform(uv.iloc[:,[3]]) # 17
# uv_ = np.concatenate((rct_no,ma_fem_dv),axis=1)
# uv_ = np.concatenate((uv_,ages),axis=1)
# uv_ = np.concatenate((uv_,zon_hlv),axis=1)
# uv = uv_

# uv, iv, tv 로 학습

In [95]:
num_users = len(user_vector)
num_items = len(item_vector)

config = {
    'num_epoch' : 200,
    'batch_size' : 256,
    'num_items' : num_items,
    'num_users' : num_users,
    'use_cuda' : True,
    'latent_dim' : 8,
}

## Dataload

In [19]:
iv

0      35
1      35
2      40
3      56
4       7
       ..
344    57
345    50
346    26
347     1
348    25
Name: clac_hlv_nm, Length: 349, dtype: int32

In [28]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

def minmax_norm(v):
    scaler = MinMaxScaler()

#     v = item_vector.iloc[:,[2]]
    scaler.fit(v)
    v_scaled= scaler.transform(v)
    return v_scaled

# transform( )시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
# iv_df = pd.DataFrame(data=iv_scaled)


In [31]:
item_vector

,Unnamed: 0,clac_mcls_nm,clac_hlv_nm
0,983,DVD/홈시어터,35
1,987,TV,35
2,1151,가공유,40
3,1699,가방세트,56
4,206,간식/안주형과자,7
...,...,...,...
344,1856,헤어케어,57
345,1563,호주산소고기,50
346,659,홈웨어,26
347,55,홍삼/인삼가공식품,1


In [39]:
item_vector['clac_hlv_nm'] = minmax_norm(item_vector.iloc[:,[2]])

In [45]:
for i in range(4):
    user_vector.iloc[:,[i+2]] = minmax_norm(user_vector.iloc[:,[i+2]])

In [46]:
user_vector

,Unnamed: 0,cust,rct_no,ma_fem_dv,ages,zon_hlv
0,0.000000,M057015266,0.000000,0.0,0.4,0.5625
1,0.000037,M919374790,0.000037,0.0,0.6,1.0000
2,0.000074,M287960590,0.000074,1.0,0.4,1.0000
3,0.000111,M763326819,0.000111,0.0,0.4,0.6250
4,0.000149,M124357021,0.000149,0.0,0.4,0.4375
...,...,...,...,...,...,...
26912,0.999851,M256907327,0.999851,1.0,0.0,1.0000
26913,0.999889,M257378621,0.999889,0.0,0.0,1.0000
26914,0.999926,M911583788,0.999926,1.0,0.2,0.8750
26915,0.999963,M911323403,0.999963,1.0,1.0,1.0000


In [17]:
data_path = './data/'
item_vector = pd.read_csv(os.path.join(data_path + 'item.csv'), encoding='utf-8')
user_vector = pd.read_csv(os.path.join(data_path + 'user.csv'), encoding='utf-8')
item_context,user_context = label_encoding(item_vector, user_vector)

#user,item,target vector
ratings_df = pd.read_csv(os.path.join(data_path + 'target.csv'), encoding='utf-8').iloc[:,1:]
ratings_df.columns = ['userId','mclsId','num']

In [53]:
ratings_df['num'] = minmax_norm(ratings_df.iloc[:,[2]])
ratings_df

,userId,mclsId,num
0,M057015266,봉지라면,0.159686
1,M057015266,탄산음료,0.526832
2,M057015266,기능성음료,0.344895
3,M057015266,모바일액세서리,0.006545
4,M057015266,커피음료,0.165576
...,...,...,...
1051177,M256907327,비스킷류,0.000000
1051178,M257378621,여성의류세트,0.000000
1051179,M911583788,스킨케어,0.000000
1051180,M911323403,밀폐/보관용기,0.000000


## ECAM_NCF(Explicit Context-aware model Neural Collaborative filltering)

### Data loader
- item,user,target,context vector 분리 및 처리해주어야 함

In [1]:
class dataloader():

    def __init__(self,data_path = './data/'):
        self.data_path = data_path

        # context vector
        item_vector = pd.read_csv(os.path.join(self.data_path + 'item.csv'), encoding='utf-8')
        user_vector = pd.read_csv(os.path.join(self.data_path + 'user.csv'), encoding='utf-8')
        item_context,user_context = self.label_encoding(item_vector, user_vector)

        #user,item,target vector
        ratings_df = pd.read_csv(os.path.join(self.data_path + 'target.csv'), encoding='utf-8').iloc[:,1:]
        ratings_df.columns = ['userId','mclsId','num']

        self.train_df, self.test_df = train_test_split(ratings_df, test_size=0.2, random_state=42, shuffle=True)

        #user vector, context
        self.users = self.train_df["userId"].unique()
        self.num_users = len(self.users)
        self.user_to_index = {user : idx for idx,user in enumerate(self.users)}
#         self.user_context = {str(idx) : user_context[user_context['cust'] == user].iloc[0,1:].to_list() for idx,user in enumerate(self.users)}
        self.user_context = {user : user_context[user_context['cust'] == user].iloc[0,2:].to_list() for idx,user in enumerate(self.users)}

        #item(mcls)vector, context
        self.mcls = self.train_df["mclsId"].unique()
        self.num_items = len(self.mcls)
        self.mcls_to_index = {mcls : idx for idx, mcls in enumerate(self.mcls)}
#         self.mcls_context = {str(idx) : item_context[item_context['clac_mcls_nm'] == mcls]['clac_hlv_nm'].item() for idx, mcls in enumerate(self.mcls)}
        self.mcls_context = {mcls : item_context[item_context['clac_mcls_nm'] == mcls]['clac_hlv_nm'].item() for idx, mcls in enumerate(self.mcls)}

        # 일반 ncf에서는 train에 있는 user,item이 test에도 있어야 하지만, context-aware이니까 굳이 이렇게 하지 않아도 될 것 같음..
#         self.test_df = self.test_df[self.test_df["userId"].isin(self.users) & self.test_df["mclsId"].isin(self.mcls)]
    
    def label_encoding(self,item_vector,user_vector):
        label = LabelEncoder()

        iv = label.fit_transform(item_vector['clac_hlv_nm'])
        item_vector['clac_hlv_nm'] = iv
        
        uv = user_vector
        uv.loc[uv['rct_no'] < 30, 'rct_no'] = 0
        uv.loc[(uv['rct_no'] >=30) & (uv['rct_no'] < 88), 'rct_no'] = 1
        uv.loc[(uv['rct_no'] >=88) & (uv['rct_no'] < 214), 'rct_no'] = 2
        uv.loc[uv['rct_no'] >=214, 'rct_no'] = 3
        uv['ma_fem_dv'] = label.fit_transform(uv['ma_fem_dv'])
        uv['ages'] = label.fit_transform(uv['ages'])
        uv['zon_hlv']=label.fit_transform(uv['zon_hlv'])

        user_vector = uv

        return item_vector, user_vector
    
    def flatten(self,arg):
        r = []
        for i in arg:
            r.extend(i) if isinstance(i,list) else r.append(i)
        return r

    def generate_trainset(self):
        x = {'user' : self.train_df["userId"].map(self.user_to_index),
            'mcls': self.train_df["mclsId"].map(self.mcls_to_index)}
        x_ = {'user_context':self.train_df['userId'].map(self.user_context),
            'mcls_context' : self.train_df['mclsId'].map(self.mcls_context)}

        x.update(x_)

        x_train = np.asarray(pd.DataFrame(x))
        x_train = np.array(list(map(self.flatten, x_train)))
        y_train = self.train_df["num"].astype(np.float32)
        return x_train, np.asarray(y_train)

    def generate_testset(self):
        x = {'user' : self.test_df["userId"].map(self.user_to_index),
            'mcls': self.test_df["mclsId"].map(self.mcls_to_index)}
        x_ = {'user_context':self.test_df['userId'].map(self.user_context),
            'mcls_context' : self.test_df['mclsId'].map(self.mcls_context)}

        x.update(x_)

        x_test = np.asarray(pd.DataFrame(x))
        x_test = np.array(list(map(self.flatten,x_test)))
        y_test = self.test_df['rating'].astype(np.float32)
        return x_test,np.asarray(y_test)
        
# if __name__ == "__main__":
#     loader = dataloader("/Users/koosup/PycharmProjects/NCF/dataset/movielens")
#     train_x,train_y = loader.generate_trainset()
#     test_x,test_y = loader.generate_testset()
#     print(train_x)
#     print(train_y)
#     print(len(train_x))
#     print(test_x)
#     print(test_y)
#     print(len(test_x))

In [2]:
d = dataloader()

NameError: name 'pd' is not defined

In [203]:
x_train,y_train = d.generate_trainset()

In [206]:
x_train.shape

(840945, 7)

In [207]:
y_train.max()

1529.0

In [208]:
x_train

array([[    0,     0,     3, ...,     1,     9,    38],
       [    1,     1,     1, ...,     3,     9,    48],
       [    2,     2,     3, ...,     3,     9,    56],
       ...,
       [ 3117,    67,     3, ...,     3,     9,    17],
       [20269,    10,     2, ...,     1,    10,    40],
       [ 1571,   194,     3, ...,     2,     9,    57]], dtype=int64)